In [ ]:
inp_dir = "/home/mrassat/STAGE/images_toulouse/crop_toulouse_2/outresults_plugin_saveinput"

config = {
    
    "input": {
        "dsm": f"{inp_dir}/dsm.tif",
        "clr": f"{inp_dir}/clr.tif",
        "classif": f"{inp_dir}/classif.tif",
        "dtm_mesh": f"{inp_dir}/one_two/out_plugin_mesh/dtm_mesh.obj",
        "dtm_mesh_epsg": 4978
    },

    "parameters": {
        "max_points": 10_000
    },

    "output": {
        "out_path": f"{inp_dir}/one_two/out_plugin_mesh/"
    }

}

In [ ]:
import rasterio as rio
import numpy as np

In [ ]:
with rio.open(config["input"]["dsm"]) as dataset:
    heights = dataset.read(1)[..., np.newaxis]

    tr = dataset.transform
    mat_tr = np.array([
        [tr[0], tr[1], tr[2]],
        [tr[3], tr[4], tr[5]],
        [    0,     0,     1]
    ])
    inv_mat_tr = np.linalg.inv(mat_tr)
    epsg = dataset.crs.to_epsg()

with rio.open(config["input"]["clr"]) as dataset:
    colors = dataset.read()
    mask = dataset.read_masks(1) != 0

with rio.open(config["input"]["classif"]) as dataset:
    classif = dataset.read()
    mask = np.logical_and(mask, classif == 0)

print(f"DSM's CRS: EPSG:{epsg}")

In [ ]:
def compute_ndvi(color):
    return (color[3] - color[0]) / (color[3] + color[0])
def compute_ndwi(color):
    return (color[3] - color[1]) / (color[3] + color[1])

indices = np.argwhere(mask[0])

ndvi = compute_ndvi( colors[:, mask[0]] )[:, np.newaxis]
ndwi = compute_ndwi( colors[:, mask[0]] )[:, np.newaxis]

ndvi_select = ( np.logical_and( ndvi > 0.5, ndwi > 0.1) ).reshape(-1)

indices = indices[ndvi_select,:]

indices = indices[ np.random.choice(
    np.arange(indices.shape[0]),
    min(indices.shape[0] // 20, config["parameters"]["max_points"])
), :]

indices = indices[:, [1, 0]]


In [ ]:
# homogeneous coords
ijs = np.pad(indices, ((0,0), (0,1)), constant_values=(1,1))

# ijs to tif crs
xys = (mat_tr @ ijs.T).T

# add Z val
xys[:,2] = heights[indices[:,1],indices[:,0],0]


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.imshow(np.moveaxis(colors[:3,:,:], (1, 2, 0), (0, 1, 2)))
plt.scatter(indices[:,0], indices[:,1], s=1)
plt.show()

In [ ]:
def project_on_dtm(points, dtm_mesh):
    """
    Projets the 2D points onto the dtm_mesh's triangles vertically (direction z)

    Returns the points with a third axis z
    """
    # points is a (n, 2) ndarray of floats

    # triangle_ids is a (m, 3) ndarray of ints
    triangle_ids = dtm_mesh["triangles"]
    # vertices is a (n, 3) ndarray of floats
    vertices = dtm_mesh["vertices"]

    min_height_achievable = np.min(dtm_mesh["vertices"][:, 2])

    nb_pts = len(points)

    # project every point vertically onto the mesh
    t_r = np.zeros(nb_pts) - 1e20
    mask = np.ones(nb_pts, dtype=bool)

    for tri in triangle_ids:

        # array of ids, used to access the overarching points
        # so as to not compute things for every vertex everytime
        over_ids = np.arange(nb_pts).astype(int)
        over_ids = over_ids[mask]

        p1 = vertices[tri[0]][:3]
        p2 = vertices[tri[1]][:3]
        p3 = vertices[tri[2]][:3]

        # vectors
        c = p3[:2] - p1[:2]
        b = p2[:2] - p1[:2]
        p = points[mask] - p1[:2]

        # dot products
        cc = np.dot(c, c)
        bc = np.dot(b, c)
        pc = np.dot(p, c)
        bb = np.dot(b, b)
        pb = np.dot(p, b)

        # barycentric coordinates
        denom = cc * bb - bc * bc
        u = (bb * pc - bc * pb) / denom
        v = (cc * pb - bc * pc) / denom

        ids_in = np.argwhere(
            np.logical_and(np.logical_and(u + v <= 1, u >= 0), v >= 0)
        )

        if len(ids_in) == 0:  # no points corresponding to this triangle
            continue

        ids_in = ids_in.reshape(-1)

        heights = (
            (p3[2] - p1[2]) * u[ids_in] + (p2[2] - p1[2]) * v[ids_in] + p1[2]
        )

        t_r[over_ids[ids_in]] = heights  # noqa: B909
        mask[over_ids[ids_in]] = False  # noqa: B909

        if mask.sum() == 0:  # all points were assigned a height! :D
            break
    
    return np.hstack((points, t_r[:, np.newaxis])), mask

In [ ]:
from cars.core import projection
import open3d as o3d

mesh_dtm = o3d.io.read_triangle_mesh( config["input"]["dtm_mesh"], False )
verts = np.asarray(mesh_dtm.vertices)
tris = np.asarray(mesh_dtm.triangles)

verts = projection.points_cloud_conversion(
    verts, config["input"]["dtm_mesh_epsg"], epsg
)

xyz_dtm, mask_good_pts = project_on_dtm(xys[:,:2], {"vertices": verts, "triangles": tris})
dz = xys[:,2] - xyz_dtm[:,2]

plt.hist(dz[dz<1000], 100)
plt.show()

In [ ]:
from cars.core import projection

xys_out_crs = projection.points_cloud_conversion(
    xys, epsg, 4326
)

out_dict = {
    "type": "FeatureCollection",
    "name": "vegetation_points",
    "features": []
}
for i, (x,y,z) in enumerate(xys_out_crs):
    
    if abs(dz[i]) > 1000:
        continue

    veg_type = "grass"
    if dz[i] > 2:
        veg_type = "bush"
    if dz[i] > 5:
        veg_type = "tree"

    out_dict["features"].append( {
        "type": "Feature",
        "properties": {
            "fid": 1,
            "DSM_4978": xyz_dtm[i,2],
            "alti": np.clip(dz[i], 0, 25),
            "vegetation_type": veg_type
        },
        "geometry": {
            "type": "Point",
            "coordinates": [ x, y ]
        }
    } )

import json
   
with open(config["output"]["out_path"]+"vegetation_points.geojson", "w") as outfile: 
    json.dump(out_dict, outfile)


print(config["output"]["out_path"]+"vegetation_points.geojson")